In [28]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
es_client = Elasticsearch('http://localhost:9200')
index_name = "homework-questions"

In [47]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [64]:
query = 'may I still enroll if the course has already started?'
#query = 'How do I execute a command in a running docker container?'
#query = 'translate English to German: How old are you?'

In [65]:
def es_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [71]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}

ANSWER: 
""".strip()
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question = doc['question'], text = doc['text']) + "\n\n"
        
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [84]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),        
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", True),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95)
    )
    return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [87]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [86]:
rag(query)
#llm(query)

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'